In [1]:
import numpy as np
import pandas as pd
import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline


In [2]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [8]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

# Comorbidities

See [here](https://www.hcup-us.ahrq.gov/toolssoftware/comorbidity/comorbidity.jsp) for more information about Elixhauser Comorbidity. This software

>assigns variables that identify comorbidities in hospital discharge records using the diagnosis coding of ICD-9-CM. 

The input data consists of diagnosis-related groups (DRG or MS-DRG) and diagnostic codes (ICD-9-CMs). 

In [12]:
query = \
"""
SELECT * FROM elixhauser_ahrq LIMIT 10;
"""
ex = pd.read_sql_query(query, con)
ex.head()

,subject_id,hadm_id,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,other_neurological,...,coagulopathy,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression
0,58526,100001,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,54610,100003,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9895,100006,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,23018,100007,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,533,100009,0,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [13]:
ex.columns

Index(['subject_id', 'hadm_id', 'congestive_heart_failure',
       'cardiac_arrhythmias', 'valvular_disease', 'pulmonary_circulation',
       'peripheral_vascular', 'hypertension', 'paralysis',
       'other_neurological', 'chronic_pulmonary', 'diabetes_uncomplicated',
       'diabetes_complicated', 'hypothyroidism', 'renal_failure',
       'liver_disease', 'peptic_ulcer', 'aids', 'lymphoma',
       'metastatic_cancer', 'solid_tumor', 'rheumatoid_arthritis',
       'coagulopathy', 'obesity', 'weight_loss', 'fluid_electrolyte',
       'blood_loss_anemia', 'deficiency_anemias', 'alcohol_abuse',
       'drug_abuse', 'psychoses', 'depression'],
      dtype='object')

# DRGCODES

>A Diagnosis-Related Group (DRG) is a statistical system of classifying any inpatient stay into groups for the purposes of payment. The DRG classification system divides possible diagnoses into more than 20 major body systems and subdivides them into almost 500 groups for the purpose of Medicare reimbursement. Factors used to determine the DRG payment amount include the diagnosis involved as well as the hospital resources necessary to treat the condition. 

In [20]:
query = \
"""
SELECT * FROM drgcodes;
"""
drg = pd.read_sql_query(query, con)
drg.head()

,row_id,subject_id,hadm_id,drg_type,drg_code,description,drg_severity,drg_mortality
0,342,2491,144486,HCFA,028,"TRAUMATIC STUPOR & COMA, COMA <1 HR AGE >17 WI...",NaN,NaN
1,343,24958,162910,HCFA,110,MAJOR CARDIOVASCULAR PROCEDURES WITH COMPLICAT...,NaN,NaN
2,344,18325,153751,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,NaN,NaN
3,345,17887,182692,HCFA,014,SPECIFIC CEREBROVASCULAR DISORDERS EXCEPT TRAN...,NaN,NaN
4,346,11113,157980,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,NaN,NaN


In [22]:
drg['description'].value_counts().head(n = 20)

Septicemia & Disseminated Infections                                        3854
NORMAL NEWBORN                                                              2805
Cardiac Valve Procedures w/o Cardiac Catheterization                        2486
Coronary Bypass w/o Cardiac Cath Or Percutaneous Cardiac Procedure          2378
Neonate, Bwt > 2499g, Normal Newborn Or Neonate W Other Problem             2099
Craniotomy Except for Trauma                                                1764
Coronary Bypass w/ Cardiac Cath Or Percutaneous Cardiac Procedure           1459
Percutaneous Cardiovascular Procedures w/ AMI                               1387
Intracranial Hemorrhage                                                     1377
Heart Failure                                                               1352
SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W MCC                          1226
Head Trauma w/ Coma > 1 Hr or Hemorrhage                                    1178
PREMATURITY WITHOUT MAJOR PR

# ESRD as a co-morbidity

ESRD is actually explicitly coded for in icd-9 as [585.6](https://www.findacode.com/icd-9/585-6-end-stage-renal-disease-icd-9-code.html). The question is if pre-existing ESRD would also be coded for. 

## Coding renal disease

See [this document](http://www.cmocares.org/documents/diagnostic-guidelines/chronic-kidney/chronic_kidney_disease_reporting1.pdf). We can also look at whether or not the patient was on dialysis. We can further ask how far the codes agree. 

## Coding pre-existing medical conditions

See [this document](https://webcache.googleusercontent.com/search?q=cache:GFuYbXEXNIIJ:https://stacks.cdc.gov/view/cdc/31381/cdc_31381_DS8.pdf+&cd=9&hl=en&ct=clnk&gl=us). 